# Workshop #2. Expression quantification

Firstly, let's create set of transcripts with different elements that would generate reads with length equal to one letter. Also let's create some random number of each transcript's molecules.

In [ ]:
import random
import numpy as np

seed = 10

transcripts = [
    "ABCDEZQ",
    "DEFHI",
    "GHIL",
    "KLONPQST",
    "PQRSTUW",
]

random.seed(seed)
n_molecules = [random.randint(30, 1000) for i in transcripts]
abundance_ground_truth = np.array(
    [n_molecules[i] / sum(n_molecules) for i in range(len(n_molecules))]
)

print("\n".join([
    f"{transcripts[i]}: {n_molecules[i]} ({(abundance_ground_truth[i] * 100):.2f}%)"
    for i in range(len(n_molecules))
]))

ABCDEZQ: 615 (26.83%)
DEFHI: 63 (2.75%)
GHIL: 469 (20.46%)
KLONPQST: 524 (22.86%)
PQRSTUW: 621 (27.09%)


Next, let's generate random reads from these transcripts.

In [ ]:
frac_reads = 0.1

random.seed(seed)
letters_pool = "".join([
    transcripts[i] * n_molecules[i]
    for i in range(len(transcripts))
])
reads = random.sample(letters_pool, int(frac_reads * len(letters_pool)))
print(f"We have {frac_reads * 100}% of transcripts in our pool ({len(reads)})")

We have 10.0% of transcripts in our pool (1503)


So, let's continue with the EM algorithm to estimate original abundance of different transcripts.

In [ ]:
def EM(transcripts, reads):
  # Works for an example with k = 1 and read_length = 1 — the simplest case

  length = np.array([len(i) for i in transcripts])
  length_factors = length / min(length)

  # Transcripts counts estimation
  def expectation(transcripts, reads, abundance):
    counts = np.zeros(len(transcripts))
    for i, read in enumerate(reads):
      equivalence_class = np.array([
          read in transcript for transcript in transcripts
      ]) # From which transcript our read might be?
      counts[equivalence_class] += (
          abundance[equivalence_class] / sum(abundance[equivalence_class])
      ) # Add fraction of the read to the transcripts according their abundances

    counts = np.round(counts, 3)
    return counts

  # Abundances maximization
  def maximization(counts, length_factors):
    abundance = (counts / sum(counts)) / length_factors # Taking into account
    # length factor
    abundance = abundance / sum(abundance)
    return abundance

  # Initianion
  abundance = np.ones(len(transcripts)) / len(transcripts) # Equal amount
  counts = expectation(transcripts, reads, abundance)

  # EM steps
  prev_counts = np.zeros(len(transcripts))
  step = 0
  while list(prev_counts) != list(counts): # Until convergence
    step += 1
    prev_counts = counts.copy()
    abundance = maximization(counts, length_factors)
    counts = expectation(transcripts, reads, abundance)

  return (abundance, counts, step)

And let's test our algorithm!

In [ ]:
abundance, counts, step = EM(transcripts, reads)

print("Differences in abundance (in %):")
print((abundance - abundance_ground_truth) * 100)

Differences in abundance (in %):
[-0.30047529 -1.00559749 -0.57102735  1.21407655  0.66302358]


In [ ]:
counts

array([422.358,  19.82 , 180.942, 438.017, 441.863])

And now, let's try to do the same calculation with `kallisto`. At the beginning, let's generate pseudo-transcripts and pseudo-reads.

In [ ]:
read_length = 50

random.seed(seed)
blocks_sequences = dict(zip(set("".join(transcripts)), [
    "".join([np.random.choice(list("ATGC")) for i in range(read_length)])
    for j in set("".join(transcripts))
]))

transcript_sequences = dict(zip(transcripts, [
    "".join([
        blocks_sequences[block] for block in transcript
    ]) for transcript in transcripts
]))

with open("reference.fasta", "w") as f:
  for transcript in transcript_sequences:
    f.write(f">{transcript}\n{transcript_sequences[transcript]}\n")

with open("reads.fastq", "w") as f:
  for i, read in enumerate(reads):
    f.write(f"@Read_{i}\n{blocks_sequences[read]}\n+\n{'I' * read_length}\n")

And let's launch `kallisto`. Firstly, let's install the tool.

In [ ]:
!git clone https://github.com/pachterlab/kallisto.git
!apt-get install autoconf
!cd kallisto && mkdir build && cd build && cmake .. && make

Second, let's generate the reference index.

In [ ]:
!./kallisto/build/src/kallisto index -i reference.idx reference.fasta


[build] loading fasta file reference.fasta
[build] k-mer length: 31
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Finished
CompactedDBG::build(): Estimated number of k-mers occurring at least once: 1120
CompactedDBG::build(): Estimated number of minimizer occurring at least once: 244
CompactedDBG::filter(): Processed 1400 k-mers in 5 reads
CompactedDBG::filter(): Found 1080 unique k-mers
CompactedDBG::filter(): Number of blocks in Bloom filter is 9
CompactedDBG::construct(): Extract approximate unitigs (1/2)
CompactedDBG::construct(): Extract approximate unitigs (2/2)
CompactedDBG::construct(): Closed all input files

CompactedDBG::construct(): Splitting unitigs (1/2)

CompactedDBG::construct(): Splitting unitigs (2/2)
CompactedDBG::construct(): Before split: 11 unitigs
CompactedDBG::construct(): After split (1/1): 11 unitigs
CompactedDBG::construct()

After, let's estimate abundances.

In [ ]:
!./kallisto/build/src/kallisto quant -i reference.idx -o results --single -l 50 -s 1 reads.fastq


[quant] fragment length distribution is truncated gaussian with mean = 50, sd = 1
[index] k-mer length: 31
[index] number of targets: 5
[index] number of k-mers: 1,080
[quant] running in single-end mode
[quant] will process file 1: reads.fastq
[quant] finding pseudoalignments for the reads ... done
[quant] processed 1,503 reads, 1,426 reads pseudoaligned
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 52 rounds



And finally, let's compare the results.

In [ ]:
import pandas as pd

kallisto_results = pd.read_csv("results/abundance.tsv", sep="\t")
kallisto_abundances = kallisto_results.tpm / kallisto_results.tpm.sum()

pd.DataFrame({
    "Difference true — kallisto (%)": list((abundance_ground_truth - kallisto_abundances) * 100),
    "Difference true — our (%)": list((abundance_ground_truth - abundance) * 100)
}, index=transcript_sequences.keys())

,Difference true — kallisto (%),Difference true — our (%)
ABCDEZQ,2.508253,0.300475
DEFHI,1.397121,1.005597
GHIL,1.836073,0.571027
KLONPQST,-3.345079,-1.214077
PQRSTUW,-2.396368,-0.663024




```
# Выбран кодовый формат
```

# Hometask

## Task 1

Please, provide an example where even EM-based algorithm estimates expression wrong. Describe in which exact cases it might happens.

Например, могут быть слишком большие каунты у одного транскрипта (выброс), или может быть шум в данных, или нет данных для транскриптов, если были ошибки в эксперименте.

In [ ]:
n_molecules = [random.randint(30, 1000) for i in transcripts]
n_molecules[2] = 10000
abundance_ground_truth = np.array(
    [n_molecules[i] / sum(n_molecules) for i in range(len(n_molecules))]
)

print("\n".join([
    f"{transcripts[i]}: {n_molecules[i]} ({(abundance_ground_truth[i] * 100):.2f}%)"
    for i in range(len(n_molecules))
]))

ABCDEZQ: 910 (7.65%)
DEFHI: 172 (1.45%)
GHIL: 10000 (84.07%)
KLONPQST: 393 (3.30%)
PQRSTUW: 420 (3.53%)


array([0.07650273, 0.01445986, 0.84068937, 0.03303909, 0.03530895])

In [ ]:
frac_reads = 0.1

random.seed(seed)
letters_pool = "".join([
    transcripts[i] * n_molecules[i]
    for i in range(len(transcripts))
])
reads = random.sample(letters_pool, int(frac_reads * len(letters_pool)))
print(f"We have {frac_reads * 100}% of transcripts in our pool ({len(reads)})")

We have 10.0% of transcripts in our pool (5331)


In [ ]:
abundance, counts, step = EM(transcripts, reads)

print("Differences in abundance (in %):")
print((abundance - abundance_ground_truth) * 100)

Differences in abundance (in %):
[-0.15933083 -0.12831402  0.22051113  0.21428775 -0.14715403]


In [ ]:
counts

array([ 623.952,   78.396, 4011.898,  334.909,  281.846])

## Task 2

Using [recount3](https://rna.recount.bio/), find any available RNA-Seq dataset (only one sample will be enough) with already estimated expressions at the level of gene counts. Re-estimate the expressions with `kallisto` and compare the results (at the gene level).

In [ ]:
# меняла среду выполнения с python на R

if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.19")

In [ ]:
BiocManager::install("recount3")

In [ ]:
library(stringr)
library(dplyr)
library(recount3)

In [ ]:
project <- create_rse_manual(
    project = "SRP033365",
    project_home = "data_sources/sra",
    organism = "human",
    annotation = "gencode_v26",
    type = "gene"
)

2024-05-31 14:26:34.098515 downloading and reading the metadata.

2024-05-31 14:26:35.264968 caching file sra.sra.SRP033365.MD.gz.

2024-05-31 14:26:35.817214 caching file sra.recount_project.SRP033365.MD.gz.

2024-05-31 14:26:36.325915 caching file sra.recount_qc.SRP033365.MD.gz.

2024-05-31 14:26:36.855448 caching file sra.recount_seq_qc.SRP033365.MD.gz.

2024-05-31 14:26:37.384561 caching file sra.recount_pred.SRP033365.MD.gz.

2024-05-31 14:26:37.493962 downloading and reading the feature information.

2024-05-31 14:26:37.926098 caching file human.gene_sums.G026.gtf.gz.

2024-05-31 14:26:38.63321 downloading and reading the counts: 1 sample across 63856 features.

2024-05-31 14:26:39.123643 caching file sra.gene_sums.SRP033365.G026.gz.

2024-05-31 14:26:39.368708 constructing the RangedSummarizedExperiment (rse) object.



In [ ]:
project

class: RangedSummarizedExperiment 
dim: 63856 1 
metadata(8): time_created recount3_version ... annotation recount3_url
assays(1): raw_counts
rownames(63856): ENSG00000278704.1 ENSG00000277400.1 ...
  ENSG00000182484.15_PAR_Y ENSG00000227159.8_PAR_Y
rowData names(10): source type ... havana_gene tag
colnames(1): SRR1039806
colData names(175): rail_id external_id ...
  recount_pred.curated.cell_line BigWigURL

In [ ]:
seqnames(rowRanges(project))

factor-Rle of length 63856 with 374 runs
  Lengths:          1          2          3 ...       2366        564
  Values : GL000009.2 GL000194.1 GL000195.1 ... chrX       chrY      
Levels(374): chr1 chr2 chr3 chr4 ... KV880765.1 KV880766.1 KV880768.1

In [ ]:
assay(project, "raw_counts") # 'http://duffel.rail.bio/recount3/human/data_sources/sra/metadata/65/SRP033365/sra.recount_seq_qc.SRP033365.MD.gz'

,SRR1039806
ENSG00000278704.1,0
ENSG00000277400.1,0
ENSG00000274847.1,0
ENSG00000277428.1,0
ENSG00000276256.1,0
ENSG00000278198.1,0
ENSG00000276932.1,0
ENSG00000273496.1,0
ENSG00000273735.4,0
ENSG00000273517.1,0


In [ ]:
rowData(project)

DataFrame with 63856 rows and 10 columns
                           source     type bp_length     phase
                         <factor> <factor> <numeric> <integer>
ENSG00000278704.1         ENSEMBL     gene      2237        NA
ENSG00000277400.1         ENSEMBL     gene      2179        NA
ENSG00000274847.1         ENSEMBL     gene      1599        NA
ENSG00000277428.1         ENSEMBL     gene       101        NA
ENSG00000276256.1         ENSEMBL     gene      2195        NA
...                           ...      ...       ...       ...
ENSG00000124334.17_PAR_Y   HAVANA     gene      2504        NA
ENSG00000185203.12_PAR_Y   HAVANA     gene      1054        NA
ENSG00000270726.6_PAR_Y    HAVANA     gene       773        NA
ENSG00000182484.15_PAR_Y   HAVANA     gene      4618        NA
ENSG00000227159.8_PAR_Y    HAVANA     gene      1306        NA
                                        gene_id              gene_type
                                    <character>            <character

In [ ]:
colData(project)

DataFrame with 1 row and 175 columns
             rail_id external_id       study sra.sample_acc.x
           <integer> <character> <character>      <character>
SRR1039806    507959  SRR1039806   SRP033365        SRS508702
           sra.experiment_acc sra.submission_acc sra.submission_center
                  <character>        <character>           <character>
SRR1039806          SRX384495          SRA114261    Cornell University
           sra.submission_lab        sra.study_title     sra.study_abstract
                    <logical>            <character>            <character>
SRR1039806                 NA Homo sapiens Transcr.. Human Transcriptome ..
           sra.study_description   sra.experiment_title sra.design_description
                       <logical>            <character>              <logical>
SRR1039806                    NA Human breast milk RN..                     NA
           sra.sample_description sra.library_name sra.library_strategy
                        <lo

In [ ]:
library("R.utils")

In [ ]:
locate_url(
  project = "SRP033365",
  project_home = "data_sources/sra",
  organism = "human",
  annotation = "gencode_v26",
  type = "metadata"
  )

sra.sra.SRP033365.MD.gz 
            "http://duffel.rail.bio/recount3/human/data_sources/sra/metadata/65/SRP033365/sra.sra.SRP033365.MD.gz" 
                                                                               sra.recount_project.SRP033365.MD.gz 
"http://duffel.rail.bio/recount3/human/data_sources/sra/metadata/65/SRP033365/sra.recount_project.SRP033365.MD.gz" 
                                                                                    sra.recount_qc.SRP033365.MD.gz 
     "http://duffel.rail.bio/recount3/human/data_sources/sra/metadata/65/SRP033365/sra.recount_qc.SRP033365.MD.gz" 
                                                                                sra.recount_seq_qc.SRP033365.MD.gz 
 "http://duffel.rail.bio/recount3/human/data_sources/sra/metadata/65/SRP033365/sra.recount_seq_qc.SRP033365.MD.gz" 
                                                                                  sra.recount_pred.SRP033365.MD.gz 
   "http://duffel.rail.bio/recount3/human/data_sources/sra/metadata/65/SRP033365/sra.recount_pred.SRP033365.MD.gz"

In [ ]:
#download.file('http://duffel.rail.bio/recount3/human/annotations/gene_sums/human.gene_sums.G026.gtf.gz', 'project.csv.gz')

In [ ]:
#gunzip('project.csv.gz')

In [ ]:
gffread